In [ ]:
%pip install --upgrade uszipcode

In [2]:
import os
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#import geopy as gp
import uszipcode as zp

/Users/sergioferreira/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
#df = pd.read_csv('inside_airbnb_zip.csv.gz', index_col = False, compression='gzip', low_memory = False)
copy_cols1 = ['id',
#            'scrape_id',
            'last_scraped',
            'host_id',
            'host_name',
#            'host_since',
#            'host_location',
            'calculated_host_listings_count',
#            'street',
            'neighbourhood_cleansed',
            'neighbourhood_group_cleansed',
#            'state',
#            'zipcode',
#            'smart_location',
#            'country',
            'latitude',
            'longitude',
#            'property_type',
            'room_type',
#            'accommodates',
#            'bathrooms',
            'bedrooms',
            'availability_365']

copy_cols2 = ['id',
            'host_id',
            'host_name',
            'calculated_host_listings_count',
            'neighbourhood',
            'neighbourhood_group',
            'latitude',
            'longitude',
            'room_type',
            'availability_365']

rename_cols2 = ['id',
            'host_id',
            'host_name',
            'calculated_host_listings_count',
            'neighbourhood_cleansed',
            'neighbourhood_group_cleansed',
            'latitude',
            'longitude',
            'room_type',
            'availability_365']

read1 = ['data/abnb_9-2022.gz','data/abnb_6-2022.gz',
        'data/abnb_3-2022.gz','data/abnb_11-2021.csv',
         'data/abnb_2015.csv','data/abnb_2016.csv',
         'data/abnb_2017.csv','data/abnb_2018.csv',
         'data/abnb_2019.csv','data/abnb_2020.csv',]

#df = pd.read_csv(, skipinitialspace=True, usecols=copy_cols, low_memory = False)

counter = 0
for item in read1:
    counter += 1
    x=0
    x2=0
    y=0
    y2=0
    if item == read1[0]:
        df1 = pd.read_csv(item, skipinitialspace=True, usecols=copy_cols1, sep=',', low_memory = False)
        df1=df1.drop_duplicates(subset='id')
#        df = df.rename(columns = rename_cols)
    elif item in read1[1:3]: #change to 4 for 2021 csv
        temp = pd.read_csv(item, skipinitialspace=True, usecols=copy_cols1, sep=',', low_memory = False)
        temp=temp.drop_duplicates(subset='id')
#        df2=df2.rename(columns = rename_cols2)
        df1 = pd.concat([df1, temp] , sort=False)
        df1 = df1.drop_duplicates(subset='id')
#        df1=df1.drop_duplicates(subset='id')
    elif item == read1[3]:
        temp = pd.read_csv(item, skipinitialspace=True, usecols=copy_cols1, sep=',', low_memory = False)
        temp=temp.drop_duplicates(subset='id')
#        df2=df2.rename(columns = rename_cols2)
        df1 = pd.concat([df1, temp] , sort=False)
#        df1=df1.drop_duplicates(subset='id')
    elif item == read1[4]:
        df2 = pd.read_csv(item, skipinitialspace=True, usecols=copy_cols2, sep=',', low_memory = False)
        df2=df2.drop_duplicates(subset='id')
        y_ins = item[10:14]
        df2['last_scraped'] = y_ins
    elif item in read1[5:10]:
        temp = pd.read_csv(item, skipinitialspace=True, usecols=copy_cols2, sep=',', low_memory = False)
        temp=temp.drop_duplicates(subset='id')
        y_ins = item[10:14]
        temp['last_scraped'] = y_ins
        df2 = pd.concat([df2, temp], sort=False)
    
#df2['zipcode'] = 'N/A'
df2['bedrooms'] = 'N/A'
df2 = df2.rename(columns={'neighbourhood_group':'neighbourhood_group_cleansed', 'neighbourhood': 'neighbourhood_cleansed'})
df2 = df2[['id', 'last_scraped', 'host_id', 'host_name', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'latitude', 'longitude', 'room_type',
       'bedrooms', 'availability_365', 'calculated_host_listings_count']]

In [4]:
df1 = pd.concat([df1,df2], sort=False)
df1 = df1.rename(columns={'neighbourhood_group_cleansed': 'borough','neighbourhood_cleansed':'neighbourhood'})

In [5]:
lat=list(df1['latitude'])
lng=list(df1['longitude'])
lat_df = pd.DataFrame(lat, columns=["latitude"])
lng_df = pd.DataFrame(lng, columns=["longitude"])

In [6]:
#search = zp.SearchEngine()
#x = len(lat)
#zlist ={}
#for i in range(0,x):
#    ilat = lat[i]
#    ilng = lng[i]
#    zippy = search.by_coordinates(ilat, ilng, radius=10, returns=1)
#    zcode = zippy[0].zipcode
#    zlist[i] = zcode

zframe = pd.read_csv('abnb_zip_indextrue.csv')

In [ ]:
#zframe = pd.DataFrame(zlist , index=[0])
#zframe = zframe.T
#zframe = zframe.rename(columns={0:'zipcode'})

In [7]:
zframe = pd.read_csv('abnb_zip_indextrue.csv')
zframe = zframe.drop(columns='Unnamed: 0')

In [8]:
coord_df = pd.DataFrame(lat, columns=["latitude"])
lng_df = pd.DataFrame(lng, columns=["longitude"])
coord_df['longitude'] = lng_df['longitude']
coord_df['geom'] = coord_df['latitude'].astype(str) + ',' + coord_df['longitude'].astype(str)
coord_df['zipcode'] = zframe['zipcode']
coord_df = coord_df.drop(columns=['latitude','longitude'])

In [9]:
df1['geom'] = df1['latitude'].astype(str) + ',' + df1['longitude'].astype(str)
df1['zipcode'] = 'N/A'
df1=df1.reset_index(drop=True)

In [10]:
df1 = pd.merge(df1, coord_df, left_index=True, right_index=True)

In [11]:
df1 = df1.drop(columns = ['zipcode_x','geom_x'])

In [12]:
df1=df1.rename(columns = {'zipcode_y':'zipcode', 'geom_y':'geom'})

In [13]:
# Using the NY_Zip_Codes.csv to filter our dataframe to only keep rows where zipcodes match one of the 5 boroughs

df1['zipcode']=df1['zipcode'].astype(int)

zips = pd.read_csv('data/NY_ZIP_Codes.csv')
zips = zips.drop(columns=['State FIPS','County Code','County FIPS', 'File Date'])

b_list= ['Bronx',
        'Richmond',
        'Queens',
        'Kings',
        'New York']

zips = zips[zips['County Name'].isin(b_list)]

df1 = df1[df1['zipcode'].isin(zips['ZIP Code'].unique())]

# To do
# Change borough values in df that don't match the zip-borough combination in zips

In [14]:
# assigns columns as date time..
# To do: Feature Engineer each column into appropriate formats

df1['last_scraped'] = pd.to_datetime(df1['last_scraped'], infer_datetime_format = True )
#df['host_since'] = pd.to_datetime(df['host_since'], infer_datetime_format = True )


# Using zips to assign borough values based on zipcode
#test_df = df.copy()
test_zips = zips.copy()

zips.loc[(test_zips['County Name'] == 'Richmond'), 'County Name'] = 'Staten Island'
zips.loc[(test_zips['County Name'] == 'New York'), 'County Name'] = 'Manhattan'
zips.loc[(test_zips['County Name'] == 'Kings'), 'County Name'] = 'Brooklyn'

zips=zips.rename(columns={'ZIP Code':'zipcode'})
zips = zips.drop_duplicates(subset='zipcode')
df1=df1.merge(zips, on='zipcode', how='left')

df1 = df1.drop(columns='borough')
df1= df1.rename(columns={'County Name':'borough'})

#df1 = df1['bedrooms'] = df1['bedrooms'].fillna('N/A')


#df.dtypes

#df['zipcode'].sort_values(ascending=False)


In [15]:
df1['bedrooms'] = df1['bedrooms'] = df1['bedrooms'].fillna('N/A')

In [16]:
df1

,id,last_scraped,host_id,host_name,neighbourhood,latitude,longitude,room_type,bedrooms,availability_365,calculated_host_listings_count,geom,zipcode,borough
0,2539,2022-09-07,2787,John,Kensington,40.64529,-73.97238,Private room,1.0,356,9,"40.64529,-73.97238",11218,Brooklyn
1,5121,2022-09-07,7356,Garon,Bedford-Stuyvesant,40.68535,-73.95512,Private room,1.0,335,2,"40.68535,-73.95512",11238,Brooklyn
2,45910,2022-09-07,204539,Mark,Ridgewood,40.70309,-73.89963,Entire home/apt,5.0,365,6,"40.70309,-73.89963",11385,Queens
3,5136,2022-09-07,7378,Rebecca,Sunset Park,40.66265,-73.99454,Entire home/apt,2.0,179,1,"40.66265,-73.99454",11215,Brooklyn
4,45935,2022-09-07,204586,L,Mott Haven,40.80635,-73.92201,Private room,1.0,83,1,"40.80635,-73.92201",10454,Bronx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335498,46937371,2020-01-01,27181542,Jt,Kips Bay,40.73961,-73.97971,Entire home/apt,N/A,180,15,"40.73961,-73.97971",11217,Brooklyn
335499,46937954,2020-01-01,287807769,Allen,Upper West Side,40.78061,-73.98514,Entire home/apt,N/A,83,1,"40.78061,-73.98514",11226,Brooklyn
335500,46938514,2020-01-01,303319010,Naiomy,East Harlem,40.80054,-73.93340,Shared room,N/A,176,1,"40.80054,-73.9334",11226,Brooklyn
335501,46938537,2020-01-01,3351317,Clara,Kips Bay,40.74033,-73.98242,Entire home/apt,N/A,83,3,"40.74033,-73.98242",10115,Manhattan


In [17]:
df1.to_csv('abnb_clean.gz')

In [ ]:
#zip_df.to_csv('abnb_zip_indextrue.csv',index=True)
#df1.to_csv('abnb_clean.gz',index=True)
#lat_df.to_csv('lat_indextrue_df.csv', index=True)
#lng_df.to_csv('lng_indextrue_df.csv', index=True)